In [123]:
import json
import requests
from bs4 import BeautifulSoup
import re

In [124]:
var_dict = {
    "Type": "type",
    "Title": "title",
    "Other Titles": "otherTitles",
    "Author(s)/Inventor(s)": "author",
    "Advisor": "advisor",
    "Co-advisor": "coAdvisor",
    "Abstract": "abstract",
    "AbstractEnglish": "abstractEnglish",
    "Keywords": "keywords",
    "Subject CNPq": "subjectCnpq",
    "Program": "program",
    "Production unit": "productionUnit",
    "Publisher": "publisher",
    "Issue Date": "issueDate",
    "Publisher country": "publisherCountry",
    "Language": "language",
    "Right access": "rightAccess",
    "Appears in Collections": "appearsInCollections",
}

def get_pdf(url):
    url = url.replace('http://hdl.handle.net/', 'https://pantheon.ufrj.br/handle/')
    response = requests.get(url)
    html = response.content
    
    soup = BeautifulSoup(html, 'html.parser')

    # acha o link do pdf usando regex pelo grupo
    regex = 'href="(\/bitstream\/.*?\.pdf)"'
    matches = re.search(regex, soup.decode())

    pdf_url = None
    if matches:
        pdf_url = 'https://pantheon.ufrj.br' + matches.group(1)

    # Extract the URL
    if pdf_url:
        print(f"PDF URL: {pdf_url}")
    else:
        print("PDF URL not found.")

    # Find the table rows
    rows = soup.find('table', class_='itemDisplayTable').find_all('tr')

    # Create a dictionary to store the data
    data_dict = {}
    abstract = False

    # Loop through rows and extract key-value pairs
    for row in rows:
        label = row.find('td', class_='metadataFieldLabel')
        value = row.find('td', class_='metadataFieldValue')
        
        if label and value:
            label_text = label.get_text(strip=True).rstrip(':')
            value_text = value.get_text(strip=True)

            if label_text == "Abstract":
                if abstract:
                    data_dict["abstractEnglish"] = value_text
                else:
                    data_dict["abstract"] = value_text
                    abstract = True
            else:
                data_dict[var_dict[label_text]] = value_text

    data_dict["pdfUrl"] = pdf_url
    return data_dict

In [125]:
data = []
num = 250

# abre doutorados.json
with open('doutorados.json') as doutorados:
    # carrega o arquivo json em um dicionário
    doutorados = json.load(doutorados)

    for doutorado in doutorados:
        if num == 0:
            break
        url = doutorado['urls'][0]
        try:
            info = get_pdf(url)
            data.append(info)
            num -= 1
        except:
            print("Erro ao obter dados do pdf")
            continue

with open('parsed_doutorados.json', 'w') as doutorados:
    # parse data to utf-8 json file
    data = json.dumps(data, indent=4, ensure_ascii=True).encode('utf8')
    doutorados.write(data.decode())


PDF URL: https://pantheon.ufrj.br/bitstream/11422/11466/3/BrunaReisLeiteFranca.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11560/1/887655.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11567/1/887747.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11570/1/887764.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11571/1/887776.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11572/1/887778.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11603/1/887791.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11604/1/885779.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11606/1/888068.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11607/1/888071.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11610/1/888193.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11622/1/886618.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11623/1/886751.pdf
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11637/1/886778.pdf
PDF URL: https://pan